In [ ]:
import openai
import os
import pandas as pd
import time
text_file = open("API_key.txt", "r")

#read whole file to a string
openai.api_key =  text_file.read()

#close file
text_file.close()


line_embeddings = []
    
def split_file_by_line(filename):
    with open(filename, 'r') as f:
        lines = f.readlines()
        line_number = 0
        for i in lines:
            line_embeddings.append([filename,i,line_number])
            line_number +=1
    return 

blocks = []
def split_file_by_character(filename, character):
    with open(filename, 'r') as f:
        lines = f.readlines()

    start_line = 0
    prev_end_line = -1
    for i, line in enumerate(lines):
        if character in line:
            prev_end_line = i
            extracted_lines = lines[start_line:prev_end_line]

            # join the extracted lines into a string
            extracted_text = "".join(extracted_lines)
            if extracted_text != "":
                blocks.append([filename,start_line,prev_end_line,extracted_text])
            #print(filename)
            start_line = i 
    
    extracted_lines = lines[prev_end_line+1:len(lines)]
    # join the extracted lines into a string
    extracted_text = "".join(extracted_lines)
    blocks.append([filename,prev_end_line+1,len(lines),extracted_text])
    
    return 


file_paths_details = [
    "index.html",
    "style.css",
    "login-page.js"
]

comment_chars = [
    "<!-",
    "/*",
    "//"
]

for i in range(0,len(file_paths_details)):
        split_file_by_line(file_paths_details[i])
        split_file_by_character(file_paths_details[i],comment_chars[i])
        

print("Total number of functions extracted:", len(split_files))

In [115]:
df = pd.DataFrame(line_embeddings)
df2 = pd.DataFrame(blocks)
df.columns = ["filepath","Code","LineNumber"]
display(df)
df2.columns = ["filepath","BlockStart","BlockStop","Code"]
display(df2)

Total number of functions extracted: 170


,filepath,Code,LineNumber
0,index.html,<!DOCTYPE html>\n,0
1,index.html,"<html lang=""en"">\n",1
2,index.html,\n,2
3,index.html,<!- File References ->\n,3
4,index.html,\n,4
...,...,...,...
129,login-page.js,location.reload();\n,12
130,login-page.js,} else {\n,13
131,login-page.js,loginErrorMsg.style.opacity = 1;\n,14
132,login-page.js,}\n,15


,filepath,BlockStart,BlockStop,Code
0,index.html,0,3,"<!DOCTYPE html>\n<html lang=""en"">\n\n"
1,index.html,3,14,<!- File References ->\n\n<head>\n <meta char...
2,index.html,14,22,"<!- Login Page Holder ->\n<main id=""main-holde..."
3,style.css,0,4,/* Page Styles */\nhtml {\n height: 100%;\n}\n
4,style.css,4,15,/* Body Styles */\nbody {\n height: 100%;\n ...
5,style.css,15,27,/* Holder for login items*/\n#main-holder {\n ...
6,style.css,27,53,/* Login Error Message Styles */\n#login-erro...


In [88]:
import random

def get_embedding(task):
    time.sleep(random.random()*8)
    response = openai.Embedding.create(
            input=task,
            model="text-embedding-ada-002"
        )
    return response['data'][0]['embedding']

df['code_embedding'] = df['Code'].apply(lambda x: get_embedding(x))
df.to_csv("code_search_openai-python.csv", index=False)


,filepath,Code,LineNumber,similarities,code_embedding
0,index.html,<!DOCTYPE html>\n,0,,"[-0.01256885752081871, 0.0028874874114990234, ..."
1,index.html,"<html lang=""en"">\n",1,,"[-0.009006879292428493, 0.010268648155033588, ..."
2,index.html,\n,2,,"[0.002454084809869528, -0.018236806616187096, ..."
3,index.html,<!- File References ->\n,3,,"[-0.025017328560352325, -0.0020156055688858032..."
4,index.html,\n,4,,"[0.0024455352686345577, -0.01827205717563629, ..."
...,...,...,...,...,...
129,login-page.js,location.reload();\n,12,,"[-0.018045900389552116, -0.0164205189794302, -..."
130,login-page.js,} else {\n,13,,"[-0.03090381622314453, -0.012458906508982182, ..."
131,login-page.js,loginErrorMsg.style.opacity = 1;\n,14,,"[-0.007993947714567184, -0.014100635424256325,..."
132,login-page.js,}\n,15,,"[-0.019705967977643013, -0.017968423664569855,..."


In [254]:
display(df)

,filepath,Code,LineNumber
0,index.html,<!DOCTYPE html>\n,0
1,index.html,"<html lang=""en"">\n",1
2,index.html,\n,2
3,index.html,<!- File References ->\n,3
4,index.html,\n,4
...,...,...,...
129,login-page.js,location.reload();\n,12
130,login-page.js,} else {\n,13
131,login-page.js,loginErrorMsg.style.opacity = 1;\n,14
132,login-page.js,}\n,15


In [91]:

def get_embedding(task):
    time.sleep(2)
    response = openai.Embedding.create(
            input=task,
            model="text-embedding-ada-002"
        )
    return response['data'][0]['embedding']

for ind in df.index:
        df['code_embedding'][ind] = get_embedding(df['Code'][ind])
print("Done")
df.to_csv("code_search_line_embeddings.csv", index=False)


/var/folders/tb/txlhvbmn2fgct223r9bxzyc80000gn/T/ipykernel_64004/2925004285.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['code_embedding'][ind] = get_embedding(df['Code'][ind])


Done


In [255]:
df = pd.read_csv('code_search_line_embeddings.csv')
df['code_embedding'] = df.code_embedding.apply(lambda x: [float(y) for y in x[1:-1].split(",")])
display(df)

,filepath,Code,LineNumber,similarities,code_embedding
0,index.html,<!DOCTYPE html>\n,0,NaN,"[-0.01256885752081871, 0.0028874874114990234, ..."
1,index.html,"<html lang=""en"">\n",1,NaN,"[-0.009006879292428493, 0.010268648155033588, ..."
2,index.html,\n,2,NaN,"[0.002454084809869528, -0.018236806616187096, ..."
3,index.html,<!- File References ->\n,3,NaN,"[-0.025017328560352325, -0.0020156055688858032..."
4,index.html,\n,4,NaN,"[0.0024455352686345577, -0.01827205717563629, ..."
...,...,...,...,...,...
129,login-page.js,location.reload();\n,12,NaN,"[-0.018045900389552116, -0.0164205189794302, -..."
130,login-page.js,} else {\n,13,NaN,"[-0.03090381622314453, -0.012458906508982182, ..."
131,login-page.js,loginErrorMsg.style.opacity = 1;\n,14,NaN,"[-0.007993947714567184, -0.014100635424256325,..."
132,login-page.js,}\n,15,NaN,"[-0.019705967977643013, -0.017968423664569855,..."


In [92]:
display(df)

,filepath,Code,LineNumber,similarities,code_embedding
0,index.html,<!DOCTYPE html>\n,0,,"[-0.01256885752081871, 0.0028874874114990234, ..."
1,index.html,"<html lang=""en"">\n",1,,"[-0.008912181481719017, 0.010274510830640793, ..."
2,index.html,\n,2,,"[0.0025078936014324427, -0.018215738236904144,..."
3,index.html,<!- File References ->\n,3,,"[-0.025017328560352325, -0.0020156055688858032..."
4,index.html,\n,4,,"[0.0025078936014324427, -0.018215738236904144,..."
...,...,...,...,...,...
129,login-page.js,location.reload();\n,12,,"[-0.018045900389552116, -0.0164205189794302, -..."
130,login-page.js,} else {\n,13,,"[-0.030943699181079865, -0.01240896712988615, ..."
131,login-page.js,loginErrorMsg.style.opacity = 1;\n,14,,"[-0.007958107627928257, -0.014057649299502373,..."
132,login-page.js,}\n,15,,"[-0.019774235785007477, -0.017938945442438126,..."


In [256]:
import numpy 
import matplotlib
task = 'Add this image in the login page holder https://blog.ipleaders.in/wp-content/uploads/2019/11/foodmitho.jpg'
from openai.embeddings_utils import cosine_similarity

def search_functions(df, code_query):
    embedding = get_embedding(code_query)
    df['similarities'] = df.code_embedding.apply(lambda x: cosine_similarity(x, embedding))
    res = df.sort_values('similarities', ascending=False).head(100)
    return res

res = search_functions(df, task)

In [258]:
res = df.sort_values('similarities', ascending=False).head(round(0.1*len(df)))
display(res)
print(len(res))

,filepath,Code,LineNumber,similarities,code_embedding
14,index.html,<!- Login Page Holder ->\n,14,0.825726,"[-0.030519431456923485, 0.023258989676833153, ..."
8,index.html,<title>Login</title>\n,8,0.805926,"[-0.001449250034056604, 0.011215200647711754, ..."
17,index.html,"<h1 id=""login-header"">Login</h1>\n",17,0.791536,"[0.005457916762679815, 0.017610518261790276, -..."
60,style.css,#login-error-msg-holder {\n,28,0.790822,"[-0.03985173627734184, 0.01832166686654091, 0...."
47,style.css,/* Holder for login items*/\n,15,0.786854,"[-0.01347746979445219, 0.019370151683688164, 0..."
26,index.html,"<input type=""submit"" value=""Login"" id=""log...",26,0.784577,"[-0.020360980182886124, -0.0023479533847421408..."
5,index.html,<head>\n,5,0.783592,"[-0.019115811213850975, 0.011791320517659187, ..."
86,style.css,#login-form {\n,54,0.782194,"[-0.003656495828181505, 0.014500855468213558, ..."
23,index.html,"<form id=""login-form"">\n",23,0.779144,"[-0.018279781565070152, 0.011316388845443726, ..."
24,index.html,"<input type=""text"" name=""username"" id=""use...",24,0.775976,"[-0.012436315417289734, -0.0007347530918195844..."


13


In [259]:
display(df2)

,filepath,BlockStart,BlockStop,Code
0,index.html,0,3,"<!DOCTYPE html>\n<html lang=""en"">\n\n"
1,index.html,3,14,<!- File References ->\n\n<head>\n <meta char...
2,index.html,14,22,"<!- Login Page Holder ->\n<main id=""main-holde..."
3,index.html,23,32,"<form id=""login-form"">\n <input type=""tex..."
4,style.css,0,4,/* Page Styles */\nhtml {\n height: 100%;\n}\n
5,style.css,4,15,/* Body Styles */\nbody {\n height: 100%;\n ...
6,style.css,15,27,/* Holder for login items*/\n#main-holder {\n ...
7,style.css,27,53,/* Login Error Message Styles */\n#login-erro...
8,style.css,54,85,#login-form {\n align-self: flex-start;\n di...
9,login-page.js,1,17,"const loginForm = document.getElementById(""log..."


In [260]:
df2["Hits"] = 0
display(df2)

,filepath,BlockStart,BlockStop,Code,Hits
0,index.html,0,3,"<!DOCTYPE html>\n<html lang=""en"">\n\n",0
1,index.html,3,14,<!- File References ->\n\n<head>\n <meta char...,0
2,index.html,14,22,"<!- Login Page Holder ->\n<main id=""main-holde...",0
3,index.html,23,32,"<form id=""login-form"">\n <input type=""tex...",0
4,style.css,0,4,/* Page Styles */\nhtml {\n height: 100%;\n}\n,0
5,style.css,4,15,/* Body Styles */\nbody {\n height: 100%;\n ...,0
6,style.css,15,27,/* Holder for login items*/\n#main-holder {\n ...,0
7,style.css,27,53,/* Login Error Message Styles */\n#login-erro...,0
8,style.css,54,85,#login-form {\n align-self: flex-start;\n di...,0
9,login-page.js,1,17,"const loginForm = document.getElementById(""log...",0


In [263]:
import numpy as np
def count_lines(filepath, start, stop):
    count = 0
    for i in df3[df3['filepath']==filepath]['LineNumber']:
        #print(i)
        for j in i:
            if(j>=start and j<stop):
                count +=1
    return count

# apply the function to each row of df2 and create a new column
df2['Hits'] = df2.apply(lambda row: count_lines(row['filepath'], row['BlockStart'], row['BlockStop']), axis=1)

In [261]:
# group by filepath and concatenate LineNumber
df3 = res.groupby("filepath").agg({"LineNumber": list}).reset_index()

display(df3)


,filepath,LineNumber
0,index.html,"[14, 8, 17, 26, 5, 23, 24, 15, 10]"
1,style.css,"[28, 15, 54, 74]"


In [264]:
display(df2)
df2.to_csv("df2.csv", index=False)


,filepath,BlockStart,BlockStop,Code,Hits
0,index.html,0,3,"<!DOCTYPE html>\n<html lang=""en"">\n\n",0
1,index.html,3,14,<!- File References ->\n\n<head>\n <meta char...,3
2,index.html,14,22,"<!- Login Page Holder ->\n<main id=""main-holde...",3
3,index.html,23,32,"<form id=""login-form"">\n <input type=""tex...",3
4,style.css,0,4,/* Page Styles */\nhtml {\n height: 100%;\n}\n,0
5,style.css,4,15,/* Body Styles */\nbody {\n height: 100%;\n ...,0
6,style.css,15,27,/* Holder for login items*/\n#main-holder {\n ...,1
7,style.css,27,53,/* Login Error Message Styles */\n#login-erro...,1
8,style.css,54,85,#login-form {\n align-self: flex-start;\n di...,2
9,login-page.js,1,17,"const loginForm = document.getElementById(""log...",0


In [247]:
df2 = df2.sort_values('Hits', ascending=False)   
print(df2.iloc[0])

filepath                                             index.html
BlockStart                                                   23
BlockStop                                                    32
Code            <form id="login-form">\n    <input type="tex...
Hits                                                          5
Name: 3, dtype: object
